In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [ ]:
classes = [line.strip() for line in open("coco.names")]
print("標籤名稱：", classes)

In [ ]:
colors = []
for i in range(len(classes)):
    colors.append((int(random.uniform(0, 255)),
                   int(random.uniform(0, 255)),
                   int(random.uniform(0, 255))))

In [ ]:
def yolo_detect(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (608, 608), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []
    
    for out in outs:
        for detection in out:
            tx, ty, tw, th, confidence = detection[0:5]
            scores = detection[5:]
            class_id = np.argmax(scores)
            if confidence>0.5:
                center_x = int(tx*width)
                center_y = int(ty*height)
                w = int(tw*width)
                h = int(th*height)
                x = int(center_x-w/2)
                y = int(center_y-h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(img, (x, y), (x+w, y+h), colors[class_ids[i]], 2)
            cv2.putText(img, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_ids[i]], 1)
    return img

In [ ]:
img = cv2.imread("elephant.jpg")
img = yolo_detect(img)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()